# Deterministic did:btc with a Update to a Singleton Beacon

This notebook works through a basic example of creating a deterministic did:btc and then updating it by publishing an update to a singleton beacon. The example uses the regtest network and interfaces with a local bitcoin node over an RPC endpoint.

# 0. Initial Setup

We do some initial setup to create a DID and an unsecuredDocument to be signed

## 0.1. Add libbtc1 python library to path

In [1]:
import sys
import os

notebooks_path = os.path.abspath(os.path.join(os.getcwd(), '..'))

# Add the Notebooks directory to the sys.path
sys.path.append(notebooks_path)

## 0.2. Create key pair

In [2]:
from buidl.mnemonic import secure_mnemonic
from buidl.hd import HDPrivateKey


In [3]:
## Run this if you want a new hardware key
# mnemonic = secure_mnemonic()

mnemonic = "prosper can dial lumber write coconut express imitate husband isolate inside release brush media please kind comic pill science repeat basic also endorse bronze"
root_hdpriv = HDPrivateKey.from_mnemonic(mnemonic, network="signet")
print("Mnemonic : ", mnemonic)

Mnemonic :  prosper can dial lumber write coconut express imitate husband isolate inside release brush media please kind comic pill science repeat basic also endorse bronze


In [4]:
didkey_purpose = "11"

initial_sk = root_hdpriv.get_private_key(didkey_purpose, address_num=2)
initial_pk = initial_sk.point

print("Secp256k1 PrivateKey", initial_sk.hex())
print("Secp256k1 Public Key", initial_pk.__repr__())

Secp256k1 PrivateKey 73fddc12fc9342bb29ae9b5ed476323bdc693a116e36512455aee830fe8a25a2
Secp256k1 Public Key S256Point(029ad5f6a85d27ee69b133aed273b4f2f5d70ed4a71675019c1a76f04c663526ef)


## 1. Deterministically Create DID BTC1 

In [5]:
from libbtc1.did import create_deterministic

did_btc1, did_document = create_deterministic(initial_pk, network="regtest")

In [6]:
did_btc1

'did:btc1:regtest:k1q2ddta4gt5n7u6d3xwhdyua57t6awrk55ut82qvurfm0qnrxx5nw7vnsy65'

In [7]:
import json
print(json.dumps(did_document, indent=2))

{
  "id": "did:btc1:regtest:k1q2ddta4gt5n7u6d3xwhdyua57t6awrk55ut82qvurfm0qnrxx5nw7vnsy65",
  "@context": [
    "https://www.w3.org/ns/did/v1",
    "https://did-btc1/TBD/context"
  ],
  "verificationMethod": [
    {
      "id": "#initialKey",
      "type": "Multikey",
      "controller": "did:btc1:regtest:k1q2ddta4gt5n7u6d3xwhdyua57t6awrk55ut82qvurfm0qnrxx5nw7vnsy65",
      "publicKeyMultibase": "z66PwJnYvwJLhGrVc8vcuUkKs99sKCzYRM2HQ2gDCGTAStHk"
    }
  ],
  "authentication": [
    "#initialKey"
  ],
  "assertionMethod": [
    "#initialKey"
  ],
  "capabilityInvocation": [
    "#initialKey"
  ],
  "capabilityDelegation": [
    "#initialKey"
  ],
  "service": [
    {
      "id": "#initial_p2pkh",
      "type": "SingletonBeacon",
      "serviceEndpoint": "bitcoin:ms4wr9JokfjLPtBVHpMefXS6domSgEdHXf"
    },
    {
      "id": "#initial_p2wpkh",
      "type": "SingletonBeacon",
      "serviceEndpoint": "bitcoin:bcrt1q06m9yn2kxgxg2mara55667958d0a6s2ksdc3dd"
    },
    {
      "id": "#initial_

## 2. Create updated DID document

- Include any additions to the DID document
- Any deletions from the DID document
- Can add/remove beacons from the service array

In [8]:
import copy
# Adding a new service
linked_domain = {
    "id":"#linked-domain",
    "type": "LinkedDomains", 
    "serviceEndpoint": "https://contact-me.com"    
}


## C2 Create DID update payload

- JSON Patch transformation from current document to new one 
- Bump version number

 
**Question** would this be better to version in the payload?
DID Document propagates to caller of resolver. That could be useful, especially for enabling resolution by version #.
https://www.w3.org/TR/did-core/#dfn-versionid 


In [9]:
# This is saying add linkedDomanSE to the end of the service list of a json document.

service_path = f"/service/{len(did_document["service"])}"

update_patch = [{'op': 'add', 'path': service_path, 'value': linked_domain}]


print(update_patch)

[{'op': 'add', 'path': '/service/3', 'value': {'id': '#linked-domain', 'type': 'LinkedDomains', 'serviceEndpoint': 'https://contact-me.com'}}]


In [10]:
import jsonpatch

patch = jsonpatch.JsonPatch(update_patch)

v2_did_document = patch.apply(did_document)


In [11]:
# This is the first update
# If you want to do another update you MUST bump the version
from buidl.helper import encode_base58, sha256
import jcs

source_hash_bytes = sha256(jcs.canonicalize(did_document))
source_hash = encode_base58(source_hash_bytes)
target_hash_bytes = sha256(jcs.canonicalize(v2_did_document))
target_hash = encode_base58(target_hash_bytes)
targetVersionId = 2


In [13]:




did_update_payload = {
    '@context': [
        'https://w3id.org/security/v2',
        'https://w3id.org/zcap/v1',
        'https://w3id.org/json-ld-patch/v1'
        # TODO did:btc1 zcap context
    ],
    'patch': update_patch,
    # TODO: this might not go here?
    'sourceHash': source_hash,
    'targetHash': target_hash,
    'targetVersionId': targetVersionId
};

## C3. Create DID Update Invocation

### C3.1 Deterministically Generate Root Capability for DID document

This is a root capability for a specific did:btc1 to invoke a capability to update that did:btc's DID document

In [14]:
import urllib
root_capability = {
  "@context": "https://w3id.org/security/v2",
  "id": f"urn:zcap:root:{urllib.parse.quote(did_btc1)}",
  "controller": did_btc1,
  "invocationTarget": did_btc1
};
root_capability

{'@context': 'https://w3id.org/security/v2',
 'id': 'urn:zcap:root:did%3Abtc1%3Aregtest%3Ak1q2ddta4gt5n7u6d3xwhdyua57t6awrk55ut82qvurfm0qnrxx5nw7vnsy65',
 'controller': 'did:btc1:regtest:k1q2ddta4gt5n7u6d3xwhdyua57t6awrk55ut82qvurfm0qnrxx5nw7vnsy65',
 'invocationTarget': 'did:btc1:regtest:k1q2ddta4gt5n7u6d3xwhdyua57t6awrk55ut82qvurfm0qnrxx5nw7vnsy65'}

### C3.2 Invoke root capability over the DID update payload

**Note:** There are no libraries for this in python. I have a separate POC that demonstrates how to achieve this using libraries for Digital Bazaar in JavaScript.

In [16]:
from di_schnorr_secp256k1.multikey import SchnorrSecp256k1Multikey
from di_schnorr_secp256k1.data_integrity_proof import DataIntegrityProof
from di_schnorr_secp256k1.cryptosuite import SchnorrSecp256k1JcsCryptoSuite

did_update_invocation = copy.deepcopy(did_update_payload)

multikey = SchnorrSecp256k1Multikey(id="#initialKey", controller=did_btc1, private_key=initial_sk)
cryptosuite = SchnorrSecp256k1JcsCryptoSuite(multikey)
di_proof = DataIntegrityProof(cryptosuite)



options = {
        "type": "DataIntegrityProof",
        "cryptosuite": "schnorr-secp256k1-jcs-2025",
        "verificationMethod": multikey.full_id(),
        "proofPurpose": "capabilityInvocation"
}


secured_did_update_payload = di_proof.add_proof(did_update_invocation, options)

# Hash invocation

Note: this might end up being an ipfs CID

In [17]:
canonical_bytes = jcs.canonicalize(secured_did_update_payload)
invocation_hash = sha256(canonical_bytes)

## Get beacon address

In [18]:
beacon_address = initial_pk.p2wpkh_address(network="regtest")
beacon_address

'bcrt1q06m9yn2kxgxg2mara55667958d0a6s2ksdc3dd'

## Fund Beacon address

Use the polar interface

In [ ]:
## TODO: YOU MUST CHANGE THIS
funding_tx_id = "e6a4c2d684edab597cb828924e3a7d34cbbc35d49501cc974797ea30ee54807c"

## TODO: MUST CHANGE. SHOULD USE BITCOIN CLI from lightning polar terminal
funding_tx_hex = "0200000000010153859e215d4dbe9dc0ef80842a3fa5bbf2b2ed9036cdc0ed656cf1edfa2bac7a0100000000fdffffff02b8b80024010000001600146cb48d4b1222cbc2f6d8e3968deac6f9770845fa00e1f505000000001600147eb6524d56320c856fa3ed29ad78b43b5fdd4156024730440220425c08d51eae5835e63ead8c5404a9c4156c1ddaf1e2fb6464fade389cb0f47b0220142e18af494683283e40272ded7b955cc037ba0ebf238ac1838ae8084668534801210286581e69b8a3e6830f7b2bed78abc392a9137bc6e94f0960f3404f8eae715d456c000000"

funding_tx = Tx.parse_hex(funding_tx_hex)

In [ ]:
funding_tx

## Construct Beacon Signal

In [ ]:
from buidl.tx import Tx, TxIn, TxOut, SIGHASH_DEFAULT

# TODO: Need to fund a beacon address
prev_tx = bytes.fromhex(funding_tx_id)  # Identifying funding tx
prev_index = 1 # Identify funding output index


tx_in = TxIn(prev_tx=prev_tx, prev_index=prev_index)

# Hack the TxIn to know about the TxOut it is spending
tx_in._script_pubkey = funding_tx.tx_outs[prev_index].script_pubkey
tx_in._value = funding_tx.tx_outs[prev_index].amount

# from buidl.tx import URL
# URL["regtest"] = "http://localhost:18443"

# print("Tx Input satoshis: ",tx_in.value(network="regtest"))

In [ ]:
from buidl.script import ScriptPubKey

script_pubkey = ScriptPubKey([0x6a, invocation_hash])

beacon_signal_txout = TxOut(0, script_pubkey)

In [ ]:
tx_fee = 350

refund_amount = tx_in.value() - tx_fee

script_pubkey = initial_pk.p2wpkh_script()
refund_out = TxOut(amount=refund_amount, script_pubkey=script_pubkey)

In [ ]:
tx_ins = [tx_in]

tx_outs = [beacon_signal_txout,refund_out]

pending_beacon_signal = Tx(version=1, tx_ins=tx_ins, tx_outs=tx_outs, network="regtest",segwit=True)


print(pending_beacon_signal.serialize().hex())

## Sign Beacon Signal

In [ ]:
pending_beacon_signal.sign_input(0, initial_sk)

In [ ]:
signed_hex = pending_beacon_signal.serialize().hex()

## Connect to local RegTest Nodew

In [ ]:
import asyncio

from bitcoinrpc import BitcoinRPC

In [ ]:
rpc = BitcoinRPC.from_config("http://localhost:18443", ("polaruser", "polarpass"))
await rpc.getconnectioncount()

In [ ]:
res = await rpc.acall("sendrawtransaction", {"hexstring": signed_hex})

In [ ]:
res